In [1]:
#IV Load the data for a station and organize for precipitation model
#Check for update
import os as os
import pandas as pd
import numpy as np
import math
from datetime import datetime
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import csv
from functools import reduce

In [2]:
try:
    usr = os.environ.get('USER') or os.environ.get('USERNAME') or getpass.getuser()
    if usr is None or str(usr).strip() == "":
        raise ValueError("username is empty")
except Exception as e:
    try:
        usr = getpass.getuser()
    except Exception:
        usr = "unknown"
    print(f"Warning: could not determine username ({e}); using '{usr}'")
print(f"Current user: {usr}")

Current user: prajwal


In [3]:
#Paths
#Need to set up based on who is running
# usr = "JH"
if usr == "prajwal":
    #ipd01 = "\\\\CXA01\\Users\\jhugh\\Documents\\HT\\NCEI_data\\2024\\"
    ipd02 = "D:\\CodeLibrary\\Python\\weathermetrics\\data\\weathermetrics\\"
    ipd03 = "D:\\CodeLibrary\\Python\\weathermetrics\\data\\weathermetrics\\"
    ipd04 = "D:\\CodeLibrary\\Python\\weathermetrics\\data\\weathermetrics\\"
    
if usr == "JH":
    #ipd01 = "\\\\CXA01\\Users\\jhugh\\Documents\\HT\\NCEI_data\\2024\\"
    #ipd02 = "\\\\CXA01\\Users\\jhugh\\Documents\\HT\\NCEI_data\\metrics_csv\\"
    ipd02 = "\\\\CXA01\\Users\\jhugh\\Documents\\Py_S4\\NCEI_Station_Info\\"
    ipd03 = "\\\\CXA01\\Users\\jhugh\\Documents\\Py_S4\\NCEI_data\\"
    ipd04 = "\\\\CXA01\\Users\\jhugh\\Documents\\Py_S4\\NCEI_parquet_files\\"


In [4]:
def fn_select_data_vectors_pr07(stid_keys,nstations,nhours,nmonths_row,nmonths_col):
    #Get the matrix of data to input into marix
    #The S4 model
    # 1. Read in combined data files for multiple stations
    # 2. Find the common dates that will be used to align the matrices
    #    Keep only the values that fit the common starting date 

    # 1. Read in data files and get start date v_date... for each
    # Check that data exist for all metrics

    df_stid = stid_keys
    df_stid2 = df_stid.iloc[:, [0]]
    df_stid2 = df_stid2.drop_duplicates() # unique()
    
    n_rows_stid = min(len(df_stid2),nstations)
    #print(f"n rows stid {n_rows_stid}")
    #Matrix in which measures are stored
    n_vmt_rows = nhours*30*max(nmonths_row,nmonths_col)
    
    data_list = []
    date_max_list =[]
    
    #1. Read in data files
    for i in range(0, n_rows_stid):
    #for i in range(0, 28):
        infile = ipd03 + str(df_stid2.iloc[i,0]) #+ "_model_data_combined.csv"
        print(infile)
        
        #Eliminate fully blank lines
        cleaned_lines = []
        with open(infile, 'r', newline='') as infile:
            reader = csv.reader(infile)
            for row in reader:
                # Example: Skip rows that are entirely empty or have only blank 'name'
                if any(cell.strip() for cell in row) and (len(row) < 1 or row[0].strip() != ''): # Assuming 'name' is the first column
                    cleaned_lines.append(row)
        
        df_metrics = pd.DataFrame(cleaned_lines[1:], columns=cleaned_lines[0]) # Assuming first row is header

        df_metrics['str_hr'] = df_metrics['nHOUR']  #.apply(lambda x: f'{x:02d}')
        df_metrics['date_hr'] = pd.to_datetime(df_metrics['Date_YYYYMMDD'] + ' ' + df_metrics['str_hr'])
        df_metrics['month_number'] = df_metrics['date_hr'].dt.month
        df_metrics['week_number'] = df_metrics['date_hr'].dt.isocalendar().week
        df_metrics['stid'] = str(df_stid2.iloc[i,0])
        data_list.append(df_metrics)

    #2. Align dates across data files
    for i in range(0, n_rows_stid):
        df = data_list[i]
        max_date = max(df['date_hr'])     
        date_max_list.append(max_date)
    
    date_filter = min(date_max_list)

    for i in range(0, n_rows_stid):
        df = data_list[i]

        filtered_df = df[df['date_hr'] < date_filter]
        filtered_df = filtered_df.iloc[(len(filtered_df) - n_vmt_rows):len(filtered_df)]
        
        idx_col = filtered_df.columns
        idx_col = str(df_stid2.iloc[i,0]) + '_' + idx_col
        filtered_df.columns = idx_col

        print(filtered_df.shape)
        
        data_list[i] = filtered_df
        
    return data_list[0:n_rows_stid]

In [5]:
def fn_create_data_matrix_pr07(lst_data,tgt_metric,nstations,nmetrics,nmonths_col,ndays_col):
    #Create a matrix for precipitation data that feeds
    #The S4 model
    # 1. Get data list and select target metric
    #Two choices now
    #    - prec
    #    - temp
    # 2.  Create output and move data into approprate cells

    # 1. get data list details and reduce to metrics
    data_vectors_list = []
    dt_rows = len(lst_data[0])
    df00 = []

    dt_rows_minus_1 = dt_rows - 1
    
    if tgt_metric == "prec": tgt_cols = [10,2,3,4,5,6,8,9,11,12]
    if tgt_metric == "temp": tgt_cols = [10,3,2,4,5,6,8,9,11,12]
    non_metrics = 4  #These are the non sensor data brought in to analysis (e.g., day of year, month, ...)
    
    #Reset the indices as all records are ordered by date and hour
    #But stations have missing records so not everything aligns
    #Need to use the target station (station 0) as the standard and left join th other fields
    print(f"number of stations in data list {len(lst_data)}")
    n_data_files = len(lst_data)-1
    
    for i in range(n_data_files):
        if i == 0:
            tdt = lst_data[i].iloc[:,tgt_cols]
        if i > 0:
            tdt = lst_data[i].iloc[:,tgt_cols[0:6]]
        tdt = tdt.iloc[::-1]
        tdt.reset_index(drop=True, inplace=True)
        first_column_name = tdt.columns[0]
        tdt.rename(columns={first_column_name: 'key_date_hr'}, inplace=True)
        #fn = f"tdt data type {tdt['key_date_hr'].dtype}"
        #print(fn)
        if tdt['key_date_hr'].dtype == "datetime64[ns]":
            df00.append(tdt)
            fn = f"1. Done with setting up station -  {i}"
            #print(fn)
    
    
    nstations_w_data = len(df00)
    
    #Take the list of station data, df00, and paste side by side
    #to produce rs which is a large matrix of data for all stations 
    join_key = 'key_date_hr'
    rs = reduce(lambda left, right: pd.merge(left, right, on=join_key, how='left'), df00)
    
    #rs has each row being a dayXhour for all stations for the stations full metrics
    fn = f"2. Shape of combined data including all stations"
    print(fn)
    print(rs.shape)
    

    #2. Create output matrix and move data into appropriate cells
    #Note: target cells get 9 elements, feature cells get 5 elements
    
    nmetrics_and_non_metrics = nmetrics + non_metrics
    fn = f"nmetrics_and_non_metrics {nmetrics_and_non_metrics}"
    print(fn)
    #OLDndataelements = nmetrics_and_non_metrics + (nstations-1)*(nmetrics)
    ndataelements = nmetrics_and_non_metrics + (nstations_w_data-1)*(nmetrics)
    fn = f"ndataelements - stations by metrics {ndataelements}"
    print(fn)
    mt_cols = 1 + ndataelements*ndays_col*24
    fn = f"mt_cols 1 + #data elements*ndays_col*24 {mt_cols}"
    print(fn)
    mt_rows = len(rs)
    
    #mt_rows = 170000  #keep here for development until done
    mt_full_set = np.empty((mt_rows,mt_cols),np.float16)
    
    #Create matrix of data
    for i in range(0,mt_rows):
        mt_full_set[i,range(0,ndataelements)] = rs.iloc[i,range(1,ndataelements+1)].apply(pd.to_numeric, errors='coerce')
    
    fn = f"3. Full mt setup with initial data for the dayXhour only"
    print(fn)  
    print(mt_full_set.shape)
    
    #Add additional column sets
    lpi = mt_rows - ndays_col*24 - 1  #number of rows in rs matrix - the data elements in a column... 
                                      #ensures all records have data
    
    #limit of rows in data set so all matrix rows have data
    lpj = ndays_col*24 -1   #number of colum sets to fill, each with 104 columns
        
    
    #The row I am operating on
    for i in range(0,lpi):
        #The columns I am going to fill
        for j in range(0,lpj):
        
            row_get = i + j + 1              #For every new column in current row we have to go down one more row
            col_get_start = 0                #data retrieved starting column 1
            col_get_end = ndataelements - 1
            
            row_put = i
            col_put_start = ndataelements*(row_get - row_put)
            col_put_end = col_put_start + ndataelements - 1
            
            mt_full_set[row_put,col_put_start:col_put_end] = mt_full_set[row_get,col_get_start:col_get_end]
    
    fn = f"4. mt_full_set with ndays of data added to the columns"
    print(fn)  
    print(mt_full_set.shape)
    
    return nstations_w_data, mt_full_set

In [6]:
def fn_create_data_matrix_DAY_SUMMARY(lst_data,tgt_metric,nstations,nmetrics,non_metrics,nmonths_col,ndays_col,ndays_to_target):
    #Create a matrix for precipitation data that feeds
    #The S4 model
    # 1. Get data list and select target metric
    #Two choices now
    #    - prec
    #    - temp
    # 2.  Create output and move data into approprate cells
    #print("HEREJHH")
    # 1. get data list details and reduce to metrics
    data_vectors_list = []
    dt_rows = len(lst_data[0])
    df00 = []

    dt_rows_minus_1 = dt_rows - 1
    
    if tgt_metric == "prec": 
        tgt_cols = [13,10,2,3,4,5,6]
        ncol_names = ['station','key_date_hr','precip','airtemp','maxRH','relhum','minRH']
    if tgt_metric == "temp": 
        tgt_cols = [13,10,3,2,4,5,6]
        ncol_names = ['station','key_date_hr','airtemp','precip','maxRH','relhum','minRH']
        
    #Reset the indices as all records are ordered by date and hour
    #But stations have missing records so not everything aligns
    #Need to use the target station (station 0) as the standard and left join th other fields
    print(f"number of stations in data list {len(lst_data)}")
    n_data_files = min(len(lst_data)-1,nstations) #TB DELETE
    n_data_files = min(len(lst_data),nstations)
    #print(f"n_data_files HERE {n_data_files}")
    
    for i in range(n_data_files):
        
        tdt = lst_data[i].iloc[:,tgt_cols]
        tdt = tdt.iloc[::-1]  #Reverses row order
        tdt.reset_index(drop=True, inplace=True)

        for j in range(len(tgt_cols)):
            curr_column_name = tdt.columns[j]
            tdt.rename(columns={curr_column_name: ncol_names[j]}, inplace=True)
        
        if tdt['key_date_hr'].dtype == "datetime64[ns]":
            
            tdt = tdt.set_index('key_date_hr')
            tdt['precip'] = pd.to_numeric(tdt['precip'],errors='coerce')
            tdt['precip'] = tdt['precip'].fillna(0)
            tdt['precip_24hr_msum'] = tdt['precip'].rolling(window='24h').sum()
            tdt['precip_24hr_mav'] = tdt['precip'].rolling(window='24h',min_periods=6).mean()
            
            tdt['airtemp'] = pd.to_numeric(tdt['airtemp'],errors='coerce')
            tdt['airtemp_24hr_mav'] = tdt['airtemp'].rolling(window='24h',min_periods=6).mean()
            
            tdt['maxRH'] = pd.to_numeric(tdt['maxRH'],errors='coerce')
            tdt['maxRH_24hr_mav'] = tdt['maxRH'].rolling(window='24h',min_periods=6).mean()
    
            tdt['relhum'] = pd.to_numeric(tdt['relhum'],errors='coerce')
            tdt['relhum_24hr_mav'] = tdt['relhum'].rolling(window='24h',min_periods=6).mean()
            
            tdt['minRH'] = pd.to_numeric(tdt['minRH'],errors='coerce')
            tdt['minRH_24hr_mav'] = tdt['minRH'].rolling(window='24h',min_periods=6).mean()        
            
            new_column_names = []
            stri = str(i)
            for n, col in enumerate(tdt.columns):
                if i < 10: 
                    prefix = f"st0{stri}_"
                    new_column_names.append(prefix + col)
                
                if i > 9: 
                    prefix = f"st{stri}_"
                    new_column_names.append(prefix + col)

            
            #print(new_column_names)    
            tdt.columns = new_column_names
            df00.append(tdt)
    
    
    df_valid_dates = pd.to_datetime(df00[0].index)
    
    print(f"Number of valid rows: {len(df_valid_dates)}")
    
    #Use a list comprehension to reindex each DataFrame, filling missing values with NaN.
    df01 = [tdt.reindex(df_valid_dates) for tdt in df00]
    df02 = [tdt.iloc[:,[0,6,7,8,9,10,11]] for tdt in df01]
        
    #Concatenate the reindexed DataFrames into a single DataFrame if needed.
    rs = pd.concat(df02, axis=1)
                
    nstations_w_data = len(df02)
    print(f"Number of stations with data: {nstations_w_data}")

    fn = f"2. Shape of combined data including all stations"
    print(fn)
    print(rs.shape)
    
    #2. Create output matrix and move data into appropriate cells
    #Note: target cells get 9 elements, feature cells get 5 elements
    
    nmetrics_and_non_metrics = nmetrics + non_metrics
    fn = f"nmetrics - {nmetrics_and_non_metrics}"
    print(fn)
    
    ndataelements = (nstations_w_data)*(nmetrics)
    fn = f"ndataelements - stations by metrics {ndataelements}"
    print(fn)
    
    rows_to_cols_sets = (ndays_col+ndays_to_target)*24
    mt_cols = ndataelements*rows_to_cols_sets   #add two columns for the month and hour for the first set of data
    fn = f"mt_cols - ndataelements*(ndays_col+ndays_to_target)*24 {mt_cols}"
    print(fn)

    #Identify the rows down to start

    rows_to_include = len(df_valid_dates) - rows_to_cols_sets - 1
    fn = f"rows to include - len(df_valid_dates) - rows_to_cols_sets - 1 {rows_to_include}"
    print(fn)
     
    mt_full_set = np.empty((rows_to_include,mt_cols),np.float16)
    fn = f"mt_shape - {mt_full_set.shape}"
    print(fn)

    ar_str_mon = np.array(rs.index.strftime('%m')).reshape(-1,1)
    ar_str_mon = ar_str_mon[0:rows_to_include]
    ar_str_hr = np.array(rs.index.strftime('%H')).reshape(-1,1)
    ar_str_hr = ar_str_hr[0:rows_to_include]
    rs.drop('st00_station', axis=1, inplace=True)
    
    #Create matrix of data
    #rows_to_include = 1000  #number of col_rows to be copied each time from column A and placed in col B    
    mt_loops = rows_to_cols_sets     #number of loops over which to copy each group of rows
    
    j = 0  #Loop across stations, set to operate only on the target station
    for i in range(0,mt_loops):
        mt_full_set[j:rows_to_include,range(i*nmetrics,i*nmetrics + nmetrics)] = rs.iloc[i:(i+rows_to_include),0:(nmetrics+1)].apply(pd.to_numeric, errors='coerce')
    
    #Add date month and hour info
    mt_full_set_rtn = np.concatenate((ar_str_mon,ar_str_hr,mt_full_set),axis=1)
    
    fn = f"3. Full mt setup with initial data for the dayXhour only"
    print(fn)  
    print(mt_full_set_rtn.shape)
    
    return mt_full_set_rtn

In [7]:
def fn_create_tgt_matrix_pr07(mtx_data,tgt_metric,tgt_mod,nstations,nmetrics,ndays_col):
    
    mtx01 = mtx_data #.iloc[0:1000,:]
    nrows = mtx01.shape[0]
    ncols = mtx01.shape[1]
    
       
    if tgt_metric == "prec" and tgt_mod == "prec_ex01": 
        rtn_matrix = fn_prec_ex01(mtx01,nstations,nmetrics,ndays_col)
    
    
    #return the matrix for analysis
    return rtn_matrix

In [8]:
def fn_create_DAY_SUMMARY_tgt_matrix_pr07(mtx_data,tgt_metric,tgt_mod,nstations,nmetrics,ndays_col,ndays_to_target):
    
    mtx01 = mtx_data  #.iloc[0:100000,:]
    nrows = mtx01.shape[0]
    ncols = mtx01.shape[1]
    
    if tgt_metric == "prec" and tgt_mod == "prec_ex02": 
        rtn_matrix = fn_prec_ex02(mtx01,1,nmetrics,ndays_col,ndays_to_target)
    
    return rtn_matrix

In [9]:
def fn_prec_ex01(mtx01,nstations,nmetrics,dayscol):
    
    df = pd.DataFrame(mtx01).reset_index(drop=True)
    ncols_orig = df.shape[1]
    print(f"Number of mxt columns {ncols_orig}")
    
    #Data come in as follows
    #target station 
    #      - number of metrics (nmetrics)
    #      - date columns (4)
    #feat station 1  - number of metrics (nmetrics)
    #feat station ...  - number of metrics (nmetrics)
    #feat station nstations  - number of metrics (nmetrics)
    #total columns = (nmetrics + date columns) + (nstations+nmetrics)
    
    offset = 4  #additional fields for target station
    data_columns_start = (nmetrics + offset) + (nstations - 1)*nmetrics
    print(f"data_columns_start {data_columns_start}")
    
    tot_per = dayscol*24  #days by 24 hours
    
    for iper in range(1,tot_per,1):

        vc_start1 = np.array([(iper*nstations*(nmetrics) + (iper*offset))])
        vc_start2 = vc_start1 + nmetrics + offset   #first set has day/month/week/hr info
        vc_end = vc_start2 + (nstations-1)*nmetrics
        vc_s2_end = np.arange(vc_start2.item(),vc_end.item(),nmetrics)
        vc_num = np.concatenate([vc_start1,vc_s2_end])
        
        nm1 = 'avg'+ str(iper)
        pds1 = df.iloc[:,vc_num].mean(axis=1,skipna=True)
        df[nm1] = pds1
        
        nm2 = 'cnt_ge_zero'+ str(iper)
        pds2 = df.iloc[:,vc_num].apply(cnt_ge0,axis=1)
        df[nm2] = pds2
        
        nm3 = 'cnt_gt_zero'+ str(iper)
        pds3 = df.iloc[:,vc_num].apply(cnt_gt0,axis=1)
        df[nm3] = pds3
               
    ncols_final = df.shape[1]
    print(f"Final shape {ncols_final}")
    new_cols = range(ncols_orig,ncols_final,1)
    print(f"New columns {new_cols}")
    old_cols = range(data_columns_start,ncols_orig-1,1)
    print(f"Old columns {old_cols}")
    vc_keep = [0]+list(new_cols) + list(old_cols)
    print(f"Final columns {ncols_final}")
    
    df.rename(columns={'0': 'tgt_precip', '5': 'day_of_year'
                       , '6': 'hour_of_day'
                       , '7': 'month_of_year'
                       , '8': 'week_of_year'}, inplace=True)
    
    df_out = pd.concat([df.iloc[:,[0,5,7,8,6]],df.iloc[:,new_cols],df.iloc[:,old_cols]],axis=1)
    
    return df_out

In [10]:
def fn_prec_ex02(mtx01,nstations,nmetrics,dayscol,ndays_to_target):
    
    df = pd.DataFrame(mtx01).reset_index(drop=True)
    ncols_orig = df.shape[1]
    print(f"Number of mxt columns {ncols_orig}")
    
    #Data come in as follows
    #target station
    #      - date columns (0,1) Month, Hour
    #      - number of metrics (6 nmetrics)
    #      - number of sets of day by hour (dayscol*24)
    

    data_columns_start = 2
    tgt_column = 3   # precip sum   
    tot_per = dayscol*24  #days by 24 hours
    
    cols = df.columns.tolist()
    ncols = [cols[2],cols[0],cols[1]] + cols[3:]
    #df_out = df.reindex(columns)
    
    
    dt_start = 2 + ndays_to_target*nmetrics*24
    
    vc = [2,0,1] + list(range(dt_start,4754))
    df_out = df.iloc[:,vc]
    
    df_out.rename(columns={'2': 'tgt_precip'
                       , '1': 'hour_of_day'
                       , '0': 'month_of_year'}, inplace=True)
                       
        
    return df_out #new_order_indices

In [11]:
#functions to identify stations in network that have rain sensor and rain
def cnt_ge0(row):
        cnt_gt_0 = sum(row >= 0)
        return cnt_gt_0

def cnt_gt0(row):
        cnt_gt_0 = sum(row > 0)
        return cnt_gt_0

In [12]:
def fn_get_nearby_stations(stid_target, target_file,in_wd):

    dt01 = target_file[target_file['tgt_stid']==stid_target]
    dt01 = dt01[['feat_stid','dist_km']]
    vc_stid = dt01['feat_stid'].to_numpy()
    
    vc_stid = np.insert(vc_stid,0,stid_target)
    
    #Need to create more combined data sets
    lst_files = os.listdir(in_wd)
    df_files = pd.DataFrame(lst_files,columns=["files"])
    df_files['stid'] = df_files['files'].str[:11]
    df_files['istarget'] = df_files['stid'].str.contains(stid_target,case=False)
    df_files['file_type'] = df_files['files'].str[23:31]
    df_files['isintarget'] = df_files['stid'].isin(vc_stid)
    
    df_files = df_files[df_files['file_type']=="combined"]
    df_files = df_files[df_files['isintarget']==True]
    
    mrg_df = pd.merge(df_files, dt01,how='left',left_on='stid',right_on='feat_stid')
    mrg_df = mrg_df.drop(columns=['feat_stid','isintarget','file_type'])
    mrg_df = mrg_df.sort_values(by='dist_km',na_position='first')
    mrg_df = mrg_df.reset_index(drop=True)
    mrg_df.columns = ['fnames','st_id','is_target_station','dist_km']
    
    return mrg_df

In [13]:
def fn_create_data_sets(tgt_stid):
    
    print(tgt_stid)
    
    #Select the target station for which predictions will be made
    dt_trg_set2 = ipd02 + "Station_Pairs_LE_100km_Info.csv"
    dt_analysis_set2 = pd.read_csv(dt_trg_set2)

    #Nearby stations within 100km
    #dt_analysis_set_keys = fn_get_nearby_stations(tgt_stid,dt_analysis_set2,ipd03)
    
    dt_analysis_set_keys = dt_analysis_set2[dt_analysis_set2['tgt_stid'] == tgt_stid]    
    print(f"dt_analysis_set_keys{dt_analysis_set_keys}")

    dt_analysis_set_keys['fpn'] = ipd03 + dt_analysis_set_keys['feat_stid']+"_model_data_combined.csv"

    dt_analysis_set_keys['file exists'] = dt_analysis_set_keys['fpn'].apply(lambda x: os.path.exists(x))

    dt_analysis_set_keys = dt_analysis_set_keys[dt_analysis_set_keys['file exists'] == True]

    nstations_in_100km = len(dt_analysis_set_keys) #number of stations within 100 km
    num_st_keep = 20   #num of stations to keep for analysis
    nstations = min(nstations_in_100km,num_st_keep)
    
    #Specify parameters governing the creation of data matrix for analysis
    #All the available stations which have all three metrics needed for model
    #vc_stid = dt_analysis_set_keys[['st_id']].drop_duplicates()

    #Organize the data based on the nature of the data so they all conform to the same size matrices
    #The metrics included are
    #  AT - air temperature in Celcius
    #  RH - relative humidity in Percent
    #  PR - precipitation in MM

    #Put them all in appropriate matrices and create list of metrics
    #Data files are of two types
    #    -  24 hour * 365 days * n years  (AT, PR)
    #    -  365 days * n years (RH)
    
    #Parameters governing data matrix
    stid_target = tgt_stid
    stid_keys = dt_analysis_set_keys

    #print(f"st id keys{dt_analysis_set_keys}")
    
    ipd = ipd02
    nhours = 24
    ndays = 365
    nmonths_row = 240  #total months on rows of data
    nmonths_col = 2 #governs the number of months predicting that are needed
    ndays_col = 3  #governs the number of days predicting that are needed
    nmetrics = 5  #number of metrics in the data file
    tgt_metric =  "prec" #"prec" "temp"    the metric to be modeled
    tgt_mod = "prec_ex01"  #specifies the metric specific approach for creating data set
    
    #1. Get the relevant station data and return in a list
    lst_data = fn_select_data_vectors_pr07(stid_keys,nstations,nhours,nmonths_row,nmonths_col)
    fn = f"std id keys {stid_keys}"
    print(fn)
    
    #2. Construct a matrix from most recent to most distant data from the list data
    nstations_w_data, mtx_data = fn_create_data_matrix_pr07(lst_data,tgt_metric,nstations,nmetrics,nmonths_col,ndays_col)
    fn = f"mtx data done"
    print(fn)
    
    #Save the mtx_data to a folder
    #This lets us start at Step 3 for subsequent runs
    print(ipd04)
    rtn_mtx_pqt = pd.DataFrame(mtx_data)
    print(rtn_mtx_pqt.shape)
    fn = f"{ipd04}{tgt_metric}_mtx_{stid_target}_stations_{nstations_w_data}_RowMn_{nmonths_row}_ColDy_{ndays_col}.parquet"
    print(fn)
    rtn_mtx_pqt.to_parquet(fn, index=False)    
    
    #3.  Create a target variable, select feature variables, create data frame for input into s4 analysis
    s4_data = fn_create_tgt_matrix_pr07(mtx_data,"prec","prec_ex01",nstations_w_data,nmetrics,ndays_col)
    
    print(f"Shape of s4_data {s4_data.shape}")
    fn = f"s4 data done"
    print(fn)
    
    #4. Save the s4_data to a folder
    #This lets us start at Step 4 for iterative runs

    print(ipd04)
    rtn_s4_data_pqt = pd.DataFrame(s4_data)
    print(rtn_s4_data_pqt.shape)
    fn = f"{ipd04}{tgt_metric}_s4data_{stid_target}_stations_{nstations_w_data}_RowMn_{nmonths_row}_ColDy_{ndays_col}.parquet"
    print(fn)
    rtn_s4_data_pqt.to_parquet(fn, index=False)    
    
    # Prep s4 data for modeling
    # Example matrix (replace this with your actual matrix)
    df = pd.DataFrame(s4_data).iloc[1:,] # Convert to DataFrame and drop the first row for binning
    print(f"All records at start {df.shape}")
    ##Only keep records w rain
    #df = df[df.iloc[:,0]>0]
    #print(df.shape)

    #Drop rows where there has been no rain in total area for past 48 hours
    ndays_check_for_rain = ndays_col
    range_of_data = ndays_check_for_rain*ndays_col*(24) + ndays_col
    vc_num = [x for x in range(5,range_of_data,3)]

    nm1 = 'sum_avg'
    pds1 = df.iloc[:,vc_num].sum(axis=1,skipna=True)
    df_area_non_zero = df[pds1 > 0]
    print(f"Records with some rain in last {ndays_check_for_rain} days {df_area_non_zero.shape}")

    #Separate into dataframes with and withot rain in past n days
    df_area_non_zero_yrain = df_area_non_zero[df_area_non_zero.iloc[:,0]>0]
    df_area_non_zero_nrain = df_area_non_zero[df_area_non_zero.iloc[:,0]==0]

    #sample non-rain to help balance outcomes if needed
    df_sample_nrain = df_area_non_zero_nrain  #df_area_non_zero_nrain.sample(n=len(df_area_non_zero_yrain))
    df_area_non_zero_ynrain = pd.concat((df_area_non_zero_yrain,df_sample_nrain),axis=0)

    #Choose the df for analysis
    dfs = df_area_non_zero_ynrain
    print(f"Full rain and no rain {dfs.shape}")

    del df, df_area_non_zero_yrain, df_area_non_zero_nrain, df_area_non_zero_ynrain
    
    # Drop rows where the first column has missing values
    # df = df.dropna(subset=[0])
    condition = np.isfinite(dfs.iloc[:,0])
    df_subset = pd.DataFrame(dfs[condition])
    del dfs
    print(df_subset.shape)


    bins = [-float('inf'),0.1,0.5,1,2,3,4,5,float('inf')]
    labels = [0,0.1, 0.5,1,2,3,4,5]
    print(bins, labels)

    df_transformed = transform_matrix_precip(df_subset, bins, labels)

    df_transformed.drop(columns=df_transformed.columns[1],inplace=True)
    df_transformed.shape

    df_transformed = df_transformed.sample(frac=1).reset_index(drop=True)

    print("Analysis Set:", df_transformed.shape)

    print(df_transformed['tgt_bin'].value_counts(dropna=False).sort_index())

    train_data, val_data, test_data = split_time_series_data(df_transformed)
    
    train_data.to_parquet(f"{ipd04}{stid_target}_train.parquet", index=False)
    val_data.to_parquet(f"{ipd04}{stid_target}_validation.parquet", index=False)
    test_data.to_parquet(f"{ipd04}{stid_target}_test.parquet", index=False)

    # code for saving to csv for testing and checking
    # train_data.to_csv(f"{ipd04}{stid_target}_train.csv", index=False)
    # val_data.to_csv(f"{ipd04}{stid_target}_validation.csv", index=False)
    # test_data.to_csv(f"{ipd04}{stid_target}_test.csv", index=False)
    
    print("Datasets exported successfully.") 
    
    return 1 #mtx_data  #stid_keys


In [14]:
def transform_matrix_precip(df, bins, labels):
    """
    Transforms the given matrix by binning the first column, dropping the first row and column,
    and adding the binned column as the first column.

    Parameters:
        matrix (pd.DataFrame): The input matrix.

    Returns:
        pd.DataFrame: The transformed matrix.
    """
    # Extract the first column (dependent variable)
    firstcolumn = df.iloc[:, 0].astype('float32')
    
    # Bin the first column
    binned_column = pd.cut(firstcolumn, bins=bins, labels=labels)
    
    # Add the binned column as the first column
    df.insert(loc=0, column='tgt_bin', value=binned_column.iloc[0:].values)
    
    # Drop columns 3 the first first column
    #df = df.drop(columns=df.columns[[3]])

    # Return the final matrix
    
    return df


In [15]:
def split_time_series_data(df, train_ratio=0.6, val_ratio=0.2, test_ratio=0.2):
    """
    Splits a time series dataset into training, validation, and test sets.

    Parameters:
        df (pd.DataFrame): The input time series dataset, ordered in descending dates.
        train_ratio (float): The proportion of data to use for training.
        val_ratio (float): The proportion of data to use for validation.
        test_ratio (float): The proportion of data to use for testing.

    Returns:
        tuple: A tuple containing three DataFrames (train, validation, test).
    """
    assert train_ratio + val_ratio + test_ratio == 1.0, "Ratios must sum to 1.0"

    # Calculate split indices
    n = len(df)
    test_end = int(n * test_ratio)    
    val_end = test_end + int(n * val_ratio)
    

    # Split the data
    test_data = df.iloc[:test_end].sort_index(ascending=False)
    val_data = df.iloc[test_end:val_end].sort_index(ascending=False)
    train_data = df.iloc[val_end:].sort_index(ascending=False)

    return train_data, val_data, test_data



In [ ]:
#DAY SUMMARY control
def stid_loop(tgt_stid,part):

    #Select the target station for which predictions will be made
    #tgt_stid =  instid #"72502014734" #instid    # "72502014734"  large file "74788012810"
    save_files = 1   #1 means save them and 0 means do not save them
    run_files = 1    #1 means do it and 0 means do not do it
    print(tgt_stid)
    #Nearby stations within 100km
    dt_trg_set2 = ipd02 + "Station_Pairs_LE_100km_Info.csv"
    dt_analysis_set2 = pd.read_csv(dt_trg_set2)

    dt_analysis_set_keys = fn_get_nearby_stations(tgt_stid,dt_analysis_set2,ipd03)
    nstations_in_100km = len(dt_analysis_set_keys) #number of stations within 100 km
    num_st_keep = 20   #num of stations to keep for analysis
    nstations = min(nstations_in_100km,num_st_keep)
    
    #dt_analysis_set_keys

    #Specify parameters governing the creation of data matrix for analysis
    #All the available stations which have all three metrics needed for model
    #vc_stid = dt_analysis_set_keys[['st_id']].drop_duplicates()

    #Organize the data based on the nature of the data so they all conform to the same size matrices
    #The metrics included are
    #  AT - air temperature in Celcius
    #  RH - relative humidity in Percent
    #  PR - precipitation in MM

    #Put them all in appropriate matrices and create list of metrics
    #Data files are of two types
    #    -  24 hour * 365 days * n years  (AT, PR)
    #    -  365 days * n years (RH)

    #Parameters governing data matrix
    stid_target = tgt_stid
    stid_keys = dt_analysis_set_keys

    ipd = ipd02
    nhours = 24
    ndays = 365
    nmonths_row = 180 #240  #total months on rows of data
    nmonths_col = 2 #governs the number of months predicting that are needed
    ndays_col = 3  #governs the number of days predicting that are needed
    nmetrics = 5  #number of metrics in the data file
    tgt_metric =  "prec" #"prec" "temp"    the metric to be modeled
    tgt_mod = "prec_ex01"  #specifies the metric specific approach for creating data set

    #2b. DAY SUMMARY ONE STATION Construct a matrix from most recent to most distant data from the list data by 
    nstations = 1
    ndays_to_target = 3
    ndays_col = 30
    nmetrics = 6
    non_metrics = 1

    #Part 1
    if part == 1:
        #Get data for target station
        lst_data = fn_select_data_vectors_pr07(stid_keys,nstations,nhours,nmonths_row,nmonths_col)
        fn = f"lst data done"
        print(fn)
        
        #Create matrix of data
        #Creates a matrix of recent to distant day hours in each row
        #Columns are
        #Col 0  - month number as string
        #Col 1  - day hour as string
        #Col 2 - Precip sum last 24 hours
        #Col 3 - Precip mean last 24 hours
        #Col 4 - Airtemp mean last 24 hours
        #Col 5 - Rel Hum max mean last 24 hours
        #Col 6 - Rel Hum mean last 24 hours
        #Col 7 - Rel Hum min mean last 24 hours
        #Cols 8 to 13 are same as 2 to 7 but for previous hour
        #Cols 14 to 19 are same as 2 to 7 but for 2 previous hours
        mtx_data = fn_create_data_matrix_DAY_SUMMARY(lst_data,tgt_metric,nstations,nmetrics,non_metrics,nmonths_col,ndays_col,ndays_to_target)
        fn = f"mtx data done - DAY SUMMARY ONE STATION"
        print(fn)
        print(mtx_data.shape)
        
        #Save the mtx_data to a folder
        print(ipd04)        
        rtn_mtx_pqt = pd.DataFrame(mtx_data)
        print(rtn_mtx_pqt.shape)
        fn = f"{ipd04}{tgt_metric}_mtx_DAY_SUMMARY_{stid_target}_stations_{nstations}_RowMn_{nmonths_row}_ColDy_{ndays_col}.parquet"
        print(fn)
        rtn_mtx_pqt.to_parquet(fn, index=False)
        
    #End part 1        

    #Part 2
    if part == 2:
        #Get the mtx_data from a folder
        #This lets us start at Step 3 for iterative runs
        fn = f"{ipd04}{tgt_metric}_mtx_DAY_SUMMARY_{stid_target}_stations_{nstations}_RowMn_{nmonths_row}_ColDy_{ndays_col}.parquet"
        rtn_mtx_data = pd.read_parquet(fn)
        #rtn_mtx_data = rtn_mtx_data_all.head(80000)
        print(rtn_mtx_data.shape)
        
        mtx_data = rtn_mtx_data.to_numpy()
        del rtn_mtx_data
        
        #DAY SUMMARY Create a target variable, select feature variables, create data frame for input into s4 analysis
        s4_data = fn_create_DAY_SUMMARY_tgt_matrix_pr07(mtx_data,"prec","prec_ex02",1,nmetrics,ndays_col,ndays_to_target)
        fn = f"s4 matrix with target data done"
        print(ipd04)
        print(fn)
        
        rtn_s4_data_pqt = pd.DataFrame(s4_data)
        print(rtn_s4_data_pqt.shape)
        fn = f"{ipd04}{tgt_metric}_s4data_DAY_SUMMARY_{stid_target}_stations_{nstations}_RowMn_{nmonths_row}_ColDy_{ndays_col}.parquet"

        print(fn)
        rtn_s4_data_pqt.to_parquet(fn, index=False)
        
    
    if part == 3:
        #This lets us start at Step 4 for iterative runs DAY SUMMARY
        fn = f"{ipd04}{tgt_metric}_s4data_DAY_SUMMARY_{stid_target}_stations_{nstations}_RowMn_{nmonths_row}_ColDy_{ndays_col}.parquet"
        print(fn)

        # Read the Parquet file into a Pandas DataFrame
        s4_data = pd.read_parquet(fn)
        print(s4_data.shape)
        
        # Prep s4 data for modeling DAY SUMMARY
        # Example matrix (replace this with your actual matrix)
        df = pd.DataFrame(s4_data).iloc[1:,] # Convert to DataFrame and drop the first row for binning
        print(f"All records at start {df.shape}")
    
        #Drop rows where there has been no rain in total area for past 48 hours
        #Choose the df for analysis
        dfs = df
        #print(f"Full rain and no rain {dfs.shape}")

        # Drop rows where the first column has missing values
        # df = df.dropna(subset=[0])
        condition = np.isfinite(dfs.iloc[:,0])
        df_subset = pd.DataFrame(dfs[condition])
        del dfs
        print(f"First missing values dropped {df_subset.shape}")

        bin_name = "bin1_5_20"
        #bins = [-float('inf'),0.1,0.5,1,2,3,4,5,float('inf')]
        #labels = [0,0.1, 0.5,1,2,3,4,5]
        bins = [-float('inf'),1,5,20,float('inf')]
        #labels = [0,1,5,20]
        labels = [0,1,2,3]
        print(bins, labels)

        df_transformed = transform_matrix_precip(df_subset, bins, labels)

        df_transformed.drop(columns=df_transformed.columns[1],inplace=True)

        df_transformed = df_transformed.sample(frac=1).reset_index(drop=True)

        print("Analysis Set:", df_transformed.shape)
        print(df_transformed['tgt_bin'].value_counts(dropna=False).sort_index())

        #Convert day and mon from char to numeric
        col_name_mon = 1
        col_name_hour = 2
        column_name_mn = df_transformed.columns[col_name_mon]
        column_name_hr = df_transformed.columns[col_name_hour]
        df_transformed[column_name_mn] = pd.to_numeric(df_transformed[column_name_mn], errors='coerce') 
        df_transformed[column_name_hr] = pd.to_numeric(df_transformed[column_name_hr], errors='coerce') 
        
        df_transformed
        
        
        #df_transformedn = df_transformed
        train_data, val_data, test_data = split_time_series_data(df_transformed)
        print("Training Set:", train_data.shape)
        print(train_data['tgt_bin'].value_counts(dropna=False).sort_index())
        print("Validation Set:", val_data.shape)
        print(val_data['tgt_bin'].value_counts(dropna=False).sort_index())
        print("Test Set:", test_data.shape)
        print(test_data['tgt_bin'].value_counts(dropna=False).sort_index())

        # Define the station ID and output directory
        # Export datasets to Parquet and CSV

        train_data.to_parquet(f"{ipd04}{stid_target}_DAYSUM_{bin_name}_train.parquet", index=False)
        val_data.to_parquet(f"{ipd04}{stid_target}_DAYSUM_{bin_name}_validation.parquet", index=False)
        test_data.to_parquet(f"{ipd04}{stid_target}_DAYSUM_{bin_name}_test.parquet", index=False)

        # code for saving to csv for testing and checking
        #train_data.to_csv(f"{ipd04}{stid_target}_DAYSUM_train.csv", index=False)
        # val_data.to_csv(f"{ipd04}{stid_target}_validation.csv", index=False)
        # test_data.to_csv(f"{ipd04}{stid_target}_test.csv", index=False)
        print("Datasets exported successfully.")

        #Code to run at cmd line
        #python -m s4model --modelname 72406093721 --trainset C:\\Users\\jhugh\\Documents\\HT\\NCEI_data\\metrics_csv\\72406093721_train.parquet --valset C:\\Users\\jhugh\\Documents\\HT\\NCEI_data\\metrics_csv\\72406093721_validation.parquet --testset C:\\Users\\jhugh\\Documents\\HT\\NCEI_data\\metrics_csv\\72406093721_test.parquet --tabulardata --dependent_variable 0 --epochs 30
        #python -m s4model --modelname 72406093721 --modeltype classification --trainset ../data/weathermetrics/74486094789_DAYSUM_bin1_5_20_train.parquet --valset ../data/weathermetrics/74486094789_DAYSUM_bin1_5_20_validation.parquet --testset ../data/weathermetrics/74486094789_DAYSUM_bin1_5_20_test.parquet --tabulardata --dependent_variable tgt_bin --epochs 30
        
        
        
    return 1 #,train_data, val_data, test_data #mtx_data #tgt_stid #s4_data

In [17]:
#jj = stid_loop("70200026617",1)

In [ ]:
#lst_stx = ["70200026617","70219026615","70316025624","72202012839","72206013889","72208013880","72214093805","72217003813","72218003820","72223013894","72241012917","72250012919","72251012924","72255012912","72312003870","72417013729","72438093819","72515004725","72519014771","72605014745","72635094860","72638094814","72639094849","72654014936","72659014929","72712014607","72734014847","72792024227"]
lst_stx = [
            # "72206013889",
            "72209864761",
            # "99769899999",
            # "72508654734",
            # "99727199999",
            # "74000154793",
           ]  
run_part = 3

def process_w_yield(my_list, run_part):
    for value in my_list:
        try:
            result = stid_loop(value, run_part)
            yield (result)
        except Exception as e:
            # yield the station id and exception so the caller can see failures
            yield (value, e)

for result in process_w_yield(lst_stx,run_part):
    print(f"Done with {result}")



In [25]:
result[3]['tgt_bin'].value_counts(dropna=False).sort_index()
result[3]['tgt_bin'][0:3]
type(result[3]['tgt_bin'])



pandas.core.series.Series

In [ ]:
runlocation = "D:/CodeLibrary/Python/weathermetrics/s4model/"
datalocation = "D:/CodeLibrary/Python/weathermetrics/data/weathermetrics/"
loglocation =  "D:/CodeLibrary/Python/weathermetrics/results/logs/"
with open(f"{runlocation}/run_models.bat", 'w') as f:
    for x in lst_stx:
        command = (
            f"python -m s4model --modelname {x} --modeltype classification "
            f"--trainset {datalocation}{x}_DAYSUM_bin1_5_20_train.parquet "
            f"--valset {datalocation}{x}_DAYSUM_bin1_5_20_validation.parquet "
            f"--testset {datalocation}{x}_DAYSUM_bin1_5_20_test.parquet "
            f"--tabulardata --dependent_variable tgt_bin --epochs 100 "
            f"> {loglocation}{x}_runlog.txt 2>&1"
        )
        f.write(command + '\n')
print("Commands exported to run_models.bat")

Commands exported to run_models.bat
